In [42]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna
import re
import gc
import time

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3
from FeatureEngineering5 import FeatureEngineering5
from models import Model1Lgb, Model1Xgb, Model1NN, Model2Linear


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('TARGET')
modules = Modules('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')
FeatureEngineering5 = FeatureEngineering5('TARGET')

In [44]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [3]:
num_rows = 30000

with FeatureEngineering5.timer("application_train and application_test"):
    df = FeatureEngineering5.get_train_test(FeatureEngineering5.DATA_DIRECTORY, num_rows= num_rows)
    print("Application dataframe shape: ", df.shape)
with FeatureEngineering5.timer("Bureau and bureau_balance data"):
    bureau_df = FeatureEngineering5.get_bureau(FeatureEngineering5.DATA_DIRECTORY, num_rows= num_rows)
    df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')
    print("Bureau dataframe shape: ", bureau_df.shape)
    del bureau_df; gc.collect()
with FeatureEngineering5.timer("previous_application"):
    prev_df = FeatureEngineering5.get_previous_applications(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
    print("Previous dataframe shape: ", prev_df.shape)
    del prev_df; gc.collect()
with FeatureEngineering5.timer("previous applications balances"):
    pos = FeatureEngineering5.get_pos_cash(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
    print("Pos-cash dataframe shape: ", pos.shape)
    del pos; gc.collect()
    ins = FeatureEngineering5.get_installment_payments(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, ins, on='SK_ID_CURR', how='left')
    print("Installments dataframe shape: ", ins.shape)
    del ins; gc.collect()
    cc = FeatureEngineering5.get_credit_card(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, cc, on='SK_ID_CURR', how='left')
    print("Credit card dataframe shape: ", cc.shape)
    del cc; gc.collect()
# Add ratios and groupby between different tables
df = FeatureEngineering5.add_ratios_features(df)
df.head()

Application dataframe shape:  (356250, 83)
application_train and application_test - done in 24s
Bureau dataframe shape:  (305811, 156)
Bureau and bureau_balance data - done in 21s
Previous dataframe shape:  (338857, 225)
previous_application - done in 28s
Pos-cash dataframe shape:  (337252, 27)
Installments dataframe shape:  (339587, 101)
Credit card dataframe shape:  (103558, 59)
previous applications balances - done in 1159s


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1.0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.374558,0.374558,2.149501,0.467976,1.175716,1.175716,0.064052,0.092379,0.951334,1.372057
1,100003,0.0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,1.584212,0.186646,15.710334,1.813930,0.290662,0.239484,0.077841,0.083552,1.098485,1.179082
2,100004,0.0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.793667,0.793667,1.566513,1.051282,0.187652,0.187652,0.042791,0.045521,3.622222,3.853333
3,100006,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.735762,0.083638,23.303080,2.120394,2.643133,1.497703,0.018185,NaN,0.113722,NaN
4,100007,0.0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.561561,0.000008,1.037195,0.558600,0.931669,0.538926,0.061350,0.057646,0.402513,0.378209


In [4]:
lgbm_categorical_feat = [
            'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
            'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']

In [5]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1.0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.374558,0.374558,2.149501,0.467976,1.175716,1.175716,0.064052,0.092379,0.951334,1.372057
1,100003,0.0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,1.584212,0.186646,15.710334,1.813930,0.290662,0.239484,0.077841,0.083552,1.098485,1.179082
2,100004,0.0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.793667,0.793667,1.566513,1.051282,0.187652,0.187652,0.042791,0.045521,3.622222,3.853333
3,100006,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.735762,0.083638,23.303080,2.120394,2.643133,1.497703,0.018185,NaN,0.113722,NaN
4,100007,0.0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.561561,0.000008,1.037195,0.558600,0.931669,0.538926,0.061350,0.057646,0.402513,0.378209


In [6]:
#### one-hotencoding実行
for col in lgbm_categorical_feat:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

df.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_TYPE_SUITE_nan,WALLSMATERIAL_MODE__1_0,WALLSMATERIAL_MODE_0_0,WALLSMATERIAL_MODE_1_0,WALLSMATERIAL_MODE_2_0,WALLSMATERIAL_MODE_3_0,WALLSMATERIAL_MODE_4_0,WALLSMATERIAL_MODE_5_0,WALLSMATERIAL_MODE_6_0,WALLSMATERIAL_MODE_nan
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,-3648.0,...,0,0,1,0,0,0,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,-1186.0,...,0,0,0,1,0,0,0,0,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,-4260.0,...,0,1,0,0,0,0,0,0,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,-9833.0,...,0,1,0,0,0,0,0,0,0,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,-4311.0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 793 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 781 features are remained after removing non-informative features
78 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 703 features are remained after removing features not interesting for LightGBM classifier
---=> 22 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 725 features

---=> df final shape: (356250, 725)  <=--- 



,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,DOCUMENT_COUNT_high_risk,DOCUMENT_COUNT_low_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_high_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,1,0,1,0,1,0,0,0,1,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,0,1,0,0,1,0,0,0,1
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,1,0,1,0,1,0,0,0,1
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,0,1,0,0,1,1,0,0,1
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,0,1,0,0,1,1,0,0,1


In [8]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 725 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 724 features are remained after removing non-informative features
19 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 705 features are remained after removing features not interesting for LightGBM classifier
---=> 9 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 714 features

---=> df final shape: (356250, 714)  <=--- 



,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,1,0,1,0,1,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,1,0,1,0,1,0,1,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,1,1,0,1,0,0,1,1,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,1,0,0,0,1,0,1,1,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,1,0,1,0,1,0,1,1,0


In [9]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,1,0,1,0,1,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,1,0,1,0,1,0,1,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,1,1,0,1,0,0,1,1,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,1,0,0,0,1,0,1,1,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,1,0,1,0,1,0,1,1,0


In [10]:
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
307506,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,1,0,0
307507,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,1,0,1,0,1,0,1,0,0
307508,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,1,0,1,0,1,0,1,0,0
307509,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,1,0,1,0,1,0,0,1,0
307510,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,1,0,0,0,0


In [11]:
df_test

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
307506,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,1,0,0
307507,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,1,0,1,0,1,0,1,0,0
307508,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,1,0,1,0,1,0,1,0,0
307509,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,1,0,1,0,1,0,0,1,0
307510,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356245,456221,121500.0,412560.0,17473.5,270000.0,0.002043,-19970,-5168.0,-9096.0,-3399,...,1,0,0,1,0,1,0,1,0,0
356246,456222,157500.0,622413.0,31909.5,495000.0,0.035797,-11186,-1149.0,-3016.0,-3003,...,0,1,0,0,0,1,0,0,0,0
356247,456223,202500.0,315000.0,33205.5,315000.0,0.026398,-15922,-3036.0,-2680.0,-1504,...,1,1,0,1,0,1,0,1,1,0
356248,456224,225000.0,450000.0,25128.0,450000.0,0.018845,-13968,-2732.0,-1461.0,-1364,...,0,1,0,1,0,1,0,0,1,0


In [12]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [13]:
df_train = df_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,1,0,1,0,0,0.033433
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,1,0,1,0,0,0.098996
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,1,0,1,0,0,0.042185
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,1,0,1,0,0,1,0,0.039429
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,1,0,0,0,1,0,0,0,0,0.161166


In [14]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [15]:
df_train = df_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,0,0.033433,0.032762
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,1,0,1,0,0,0.098996,0.100620
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,1,0,1,0,0,0.042185,0.044999
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,1,0,0,1,0,0.039429,0.034211
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,1,0,0,0,0,0.161166,0.172532


In [16]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [17]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,1,0,0,0.033433,0.032762,0.040068
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,1,0,0,0.098996,0.100620,0.095240
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,1,0,0,0.042185,0.044999,0.044435
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,1,0,0,1,0,0.039429,0.034211,0.036586
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,1,0,0,0,0,0.161166,0.172532,0.141754


In [18]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [19]:
df_train = df_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,0,0.033433,0.032762,0.040068,0.035234
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,1,0,0,0.098996,0.100620,0.095240,0.107426
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,1,0,0,0.042185,0.044999,0.044435,0.048029
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,0,1,0,0.039429,0.034211,0.036586,0.032975
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [20]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [21]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,1,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045


In [22]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [23]:
df_train = df_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442


In [24]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [25]:
df_train = df_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637


In [26]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [27]:
df_train = df_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295


In [28]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [29]:
df_train = df_train.merge(train_set4a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048


In [30]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [31]:
df_train = df_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113,0.034978
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192,0.118268
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271,0.039774
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206,0.037283
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048,0.164890


In [32]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.259405,0.218256,0.207998
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.023080,0.012283,0.013902
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033290,0.033160,0.034973
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.022860,0.029616,0.040586
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.033131,0.035875,0.036123


In [33]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113,0.034978
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192,0.118268
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271,0.039774
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206,0.037283
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048,0.164890


In [34]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [35]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,-2120,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.259405,0.218256,0.207998
100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,-291,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.023080,0.012283,0.013902
100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,-2531,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033290,0.033160,0.034973
100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,-2437,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.022860,0.029616,0.040586
100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,-3458,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.033131,0.035875,0.036123


In [36]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,0.579149,...,2.106458,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.888223,1.449139,1.374899
100003,0.0,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,1.790846,...,-0.598379,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.587171,-0.715210,-0.685605
100004,0.0,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,0.306866,...,-0.501560,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.480227,-0.495838,-0.461924
100006,0.0,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,0.369140,...,-0.541154,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.589469,-0.533075,-0.402327
100007,0.0,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,-0.307264,...,-0.596392,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.481891,-0.467308,-0.449710


In [37]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,2.106458,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.888223,1.449139,1.374899
1,100003,0.0,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,-0.598379,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.587171,-0.715210,-0.685605
2,100004,0.0,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-0.501560,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.480227,-0.495838,-0.461924
3,100006,0.0,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-0.541154,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.589469,-0.533075,-0.402327
4,100007,0.0,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-0.596392,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.481891,-0.467308,-0.449710


In [38]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 764.82 MB
Memory usage of optimization is 890537504.00 MB
Decreased by -116436865.0%
Memory usage of dataframe is 121.14 MB
Memory usage of optimization is 140967776.00 MB
Decreased by -116365268.0%


,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,0.379883,...,2.107422,1.634766,1.896484,1.301758,1.757812,1.261719,1.746094,1.888672,1.449219,1.375000
1,100003,0.0,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,1.079102,...,-0.598145,-0.587891,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.587402,-0.715332,-0.685547
2,100004,0.0,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,0.206055,...,-0.501465,-0.423340,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.480225,-0.495850,-0.461914
3,100006,0.0,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,-1.375000,...,-0.541016,-0.472900,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.589355,-0.533203,-0.402344
4,100007,0.0,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,0.191284,...,-0.596191,-0.537598,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.481934,-0.467285,-0.449707


In [39]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [40]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [46]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Lgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[LightGBM] [Info] Number of positive: 18633, number of negative: 211996
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062808 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 204565
[LightGBM] [Info] Number of data points in the train set: 230629, number of used features: 720
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080792 -> initscore=-2.431633
[LightGBM] [Info] Start training from score -2.431633
[1]	train's auc: 0.785599	valid's auc: 0.784132
[2]	train's auc: 0.778841	valid's auc: 0.776739
[3]	train's auc: 0.774621	valid's auc: 0.771274
[4]	train's auc: 0.787178	valid's auc: 0.783662
[5]	train's auc: 0.786091	valid's auc: 0.781317
[6]	train's auc: 0.783149	valid's auc: 0.777947
[7]	train's auc: 0.782763	valid's auc: 0.776813
[8]	train's auc: 0.788164	valid's auc: 0.782753
[9]	train's auc: 0.791762	valid's auc: 0.786872
[1

[157]	train's auc: 0.80368	valid's auc: 0.794811
[158]	train's auc: 0.803722	valid's auc: 0.794829
[159]	train's auc: 0.803757	valid's auc: 0.794831
[160]	train's auc: 0.803809	valid's auc: 0.794802
[161]	train's auc: 0.80385	valid's auc: 0.794814
[162]	train's auc: 0.803882	valid's auc: 0.794829
[163]	train's auc: 0.803913	valid's auc: 0.794854
[164]	train's auc: 0.803944	valid's auc: 0.794854
[165]	train's auc: 0.804001	valid's auc: 0.794876
[166]	train's auc: 0.804045	valid's auc: 0.794877
[167]	train's auc: 0.804079	valid's auc: 0.794887
[168]	train's auc: 0.8041	valid's auc: 0.794901
[169]	train's auc: 0.804135	valid's auc: 0.794909
[170]	train's auc: 0.80418	valid's auc: 0.794923
[171]	train's auc: 0.80423	valid's auc: 0.794967
[172]	train's auc: 0.804286	valid's auc: 0.79496
[173]	train's auc: 0.804328	valid's auc: 0.794976
[174]	train's auc: 0.804361	valid's auc: 0.794977
[175]	train's auc: 0.804408	valid's auc: 0.794999
[176]	train's auc: 0.804436	valid's auc: 0.795006
[177]	t

[323]	train's auc: 0.810742	valid's auc: 0.795662
[324]	train's auc: 0.810782	valid's auc: 0.795661
[325]	train's auc: 0.810828	valid's auc: 0.795664
[326]	train's auc: 0.810863	valid's auc: 0.795663
[327]	train's auc: 0.810903	valid's auc: 0.795661
[328]	train's auc: 0.81095	valid's auc: 0.795673
[329]	train's auc: 0.810996	valid's auc: 0.795663
[330]	train's auc: 0.81104	valid's auc: 0.795657
[331]	train's auc: 0.811079	valid's auc: 0.795657
[332]	train's auc: 0.811109	valid's auc: 0.795656
[333]	train's auc: 0.811148	valid's auc: 0.795651
[334]	train's auc: 0.811187	valid's auc: 0.795647
[335]	train's auc: 0.811231	valid's auc: 0.795652
[336]	train's auc: 0.811267	valid's auc: 0.79565
[337]	train's auc: 0.811311	valid's auc: 0.795658
[338]	train's auc: 0.81135	valid's auc: 0.79565
[339]	train's auc: 0.811392	valid's auc: 0.795644
[340]	train's auc: 0.811423	valid's auc: 0.795646
[341]	train's auc: 0.811463	valid's auc: 0.795639
[342]	train's auc: 0.811506	valid's auc: 0.795657
[343]

[490]	train's auc: 0.81752	valid's auc: 0.795554
[491]	train's auc: 0.817556	valid's auc: 0.795554
[492]	train's auc: 0.817591	valid's auc: 0.795552
[493]	train's auc: 0.817629	valid's auc: 0.795552
[494]	train's auc: 0.817673	valid's auc: 0.795554
[495]	train's auc: 0.817713	valid's auc: 0.795559
[496]	train's auc: 0.817749	valid's auc: 0.795554
[497]	train's auc: 0.817782	valid's auc: 0.795552
[498]	train's auc: 0.817821	valid's auc: 0.795567
[499]	train's auc: 0.81786	valid's auc: 0.795564
[500]	train's auc: 0.817894	valid's auc: 0.795557
[501]	train's auc: 0.817935	valid's auc: 0.795551
[502]	train's auc: 0.817971	valid's auc: 0.795554
[503]	train's auc: 0.818012	valid's auc: 0.795561
[504]	train's auc: 0.818062	valid's auc: 0.795562
[505]	train's auc: 0.818086	valid's auc: 0.795561
[506]	train's auc: 0.818127	valid's auc: 0.795557
[507]	train's auc: 0.818169	valid's auc: 0.795556
[508]	train's auc: 0.818208	valid's auc: 0.795555
[509]	train's auc: 0.818247	valid's auc: 0.795546
[5

[656]	train's auc: 0.823779	valid's auc: 0.795293
[657]	train's auc: 0.823813	valid's auc: 0.79529
[658]	train's auc: 0.823848	valid's auc: 0.795281
[659]	train's auc: 0.823886	valid's auc: 0.795273
[660]	train's auc: 0.823913	valid's auc: 0.79527
[661]	train's auc: 0.823947	valid's auc: 0.795263
[662]	train's auc: 0.823986	valid's auc: 0.795262
[663]	train's auc: 0.82402	valid's auc: 0.795266
[664]	train's auc: 0.824049	valid's auc: 0.795273
[665]	train's auc: 0.824087	valid's auc: 0.795272
[666]	train's auc: 0.824121	valid's auc: 0.795265
[667]	train's auc: 0.824156	valid's auc: 0.795264
[668]	train's auc: 0.824191	valid's auc: 0.795265
[669]	train's auc: 0.824233	valid's auc: 0.795266
[670]	train's auc: 0.824264	valid's auc: 0.79526
[671]	train's auc: 0.824299	valid's auc: 0.79526
[672]	train's auc: 0.824331	valid's auc: 0.795265
[673]	train's auc: 0.824369	valid's auc: 0.795263
[674]	train's auc: 0.824411	valid's auc: 0.795263
[675]	train's auc: 0.824454	valid's auc: 0.795265
[676]

...

[1219]	train's auc: 0.842686	valid's auc: 0.791872
[1220]	train's auc: 0.84272	valid's auc: 0.791875
[1221]	train's auc: 0.842752	valid's auc: 0.791874
[1222]	train's auc: 0.84278	valid's auc: 0.791871
[1223]	train's auc: 0.842808	valid's auc: 0.791866
[1224]	train's auc: 0.842833	valid's auc: 0.791861
[1225]	train's auc: 0.842852	valid's auc: 0.791857
[1226]	train's auc: 0.842879	valid's auc: 0.791856
[1227]	train's auc: 0.8429	valid's auc: 0.79186
[1228]	train's auc: 0.842929	valid's auc: 0.791863
[1229]	train's auc: 0.842963	valid's auc: 0.791856
[1230]	train's auc: 0.842992	valid's auc: 0.791852
[1231]	train's auc: 0.843024	valid's auc: 0.791848
[1232]	train's auc: 0.843054	valid's auc: 0.791841
[1233]	train's auc: 0.843077	valid's auc: 0.791841
[1234]	train's auc: 0.843104	valid's auc: 0.791836
[1235]	train's auc: 0.843129	valid's auc: 0.791837
[1236]	train's auc: 0.843164	valid's auc: 0.791842
[1237]	train's auc: 0.843203	valid's auc: 0.791841
[1238]	train's auc: 0.843233	valid's

[1383]	train's auc: 0.847385	valid's auc: 0.791626
[1384]	train's auc: 0.847411	valid's auc: 0.791619
[1385]	train's auc: 0.847433	valid's auc: 0.791613
[1386]	train's auc: 0.847455	valid's auc: 0.791616
[1387]	train's auc: 0.847488	valid's auc: 0.791619
[1388]	train's auc: 0.847519	valid's auc: 0.791621
[1389]	train's auc: 0.847577	valid's auc: 0.791611
[1390]	train's auc: 0.847604	valid's auc: 0.791611
[1391]	train's auc: 0.847627	valid's auc: 0.791606
[1392]	train's auc: 0.84765	valid's auc: 0.791606
[1393]	train's auc: 0.847673	valid's auc: 0.791596
[1394]	train's auc: 0.847693	valid's auc: 0.791595
[1395]	train's auc: 0.847716	valid's auc: 0.79159
[1396]	train's auc: 0.847738	valid's auc: 0.791588
[1397]	train's auc: 0.847762	valid's auc: 0.791596
[1398]	train's auc: 0.847789	valid's auc: 0.791588
[1399]	train's auc: 0.847818	valid's auc: 0.791586
[1400]	train's auc: 0.847873	valid's auc: 0.791585
[1401]	train's auc: 0.847897	valid's auc: 0.791582
[1402]	train's auc: 0.847918	vali

[1545]	train's auc: 0.851865	valid's auc: 0.791498
[1546]	train's auc: 0.851887	valid's auc: 0.791494
[1547]	train's auc: 0.851909	valid's auc: 0.791496
[1548]	train's auc: 0.851934	valid's auc: 0.791497
[1549]	train's auc: 0.851962	valid's auc: 0.791502
[1550]	train's auc: 0.851985	valid's auc: 0.791499
[1551]	train's auc: 0.85201	valid's auc: 0.791503
[1552]	train's auc: 0.852033	valid's auc: 0.791501
[1553]	train's auc: 0.852066	valid's auc: 0.791493
[1554]	train's auc: 0.8521	valid's auc: 0.7915
[1555]	train's auc: 0.852126	valid's auc: 0.791499
[1556]	train's auc: 0.85215	valid's auc: 0.791501
[1557]	train's auc: 0.852173	valid's auc: 0.791503
[1558]	train's auc: 0.852202	valid's auc: 0.791505
[1559]	train's auc: 0.852224	valid's auc: 0.791497
[1560]	train's auc: 0.852243	valid's auc: 0.7915
[1561]	train's auc: 0.852264	valid's auc: 0.791498
[1562]	train's auc: 0.852276	valid's auc: 0.7915
[1563]	train's auc: 0.852303	valid's auc: 0.7915
[1564]	train's auc: 0.852329	valid's auc: 0

[1708]	train's auc: 0.856254	valid's auc: 0.791425
[1709]	train's auc: 0.856268	valid's auc: 0.791422
[1710]	train's auc: 0.856295	valid's auc: 0.791435
[1711]	train's auc: 0.856322	valid's auc: 0.791422
[1712]	train's auc: 0.856345	valid's auc: 0.791424
[1713]	train's auc: 0.856369	valid's auc: 0.791434
[1714]	train's auc: 0.856393	valid's auc: 0.791435
[1715]	train's auc: 0.856413	valid's auc: 0.791439
[1716]	train's auc: 0.856438	valid's auc: 0.791438
[1717]	train's auc: 0.856464	valid's auc: 0.791431
[1718]	train's auc: 0.856495	valid's auc: 0.791419
[1719]	train's auc: 0.856533	valid's auc: 0.79142
[1720]	train's auc: 0.856549	valid's auc: 0.791421
[1721]	train's auc: 0.856573	valid's auc: 0.791422
[1722]	train's auc: 0.856624	valid's auc: 0.791421
[1723]	train's auc: 0.856638	valid's auc: 0.79143
[1724]	train's auc: 0.856664	valid's auc: 0.791429
[1725]	train's auc: 0.856693	valid's auc: 0.791432
[1726]	train's auc: 0.856714	valid's auc: 0.791431
[1727]	train's auc: 0.856732	vali

[1872]	train's auc: 0.860376	valid's auc: 0.791279
[1873]	train's auc: 0.8604	valid's auc: 0.791287
[1874]	train's auc: 0.860417	valid's auc: 0.79129
[1875]	train's auc: 0.860439	valid's auc: 0.791284
[1876]	train's auc: 0.860454	valid's auc: 0.791286
[1877]	train's auc: 0.860495	valid's auc: 0.791312
[1878]	train's auc: 0.860549	valid's auc: 0.791313
[1879]	train's auc: 0.860563	valid's auc: 0.791318
[1880]	train's auc: 0.860587	valid's auc: 0.791328
[1881]	train's auc: 0.860612	valid's auc: 0.791327
[1882]	train's auc: 0.860638	valid's auc: 0.791325
[1883]	train's auc: 0.860659	valid's auc: 0.791328
[1884]	train's auc: 0.860709	valid's auc: 0.79133
[1885]	train's auc: 0.860748	valid's auc: 0.791326
[1886]	train's auc: 0.860768	valid's auc: 0.791324
[1887]	train's auc: 0.860799	valid's auc: 0.791324
[1888]	train's auc: 0.860819	valid's auc: 0.791324
[1889]	train's auc: 0.860856	valid's auc: 0.791331
[1890]	train's auc: 0.86088	valid's auc: 0.791323
[1891]	train's auc: 0.860901	valid's

In [47]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307506

In [49]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set5a': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set5a
0,100002,0.196429
1,100003,0.013261
2,100004,0.030715
3,100006,0.034176
4,100007,0.038256


In [50]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set5a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [51]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [52]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set5a': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set5a
0,100001,0.034370
1,100005,0.116919
2,100013,0.037191
3,100028,0.051958
4,100038,0.141608


In [53]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set5a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)